In [1]:
import tensorflow as tf
from tensorflow import keras

In [27]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


## 그래디언트 폭주나 소실을 막기 위한 가중치 초기화
## 초기화 전략-활성화 함수-정규분포의 분산
글로럿 - 활성화 함수 없음, tanh,sigmoid, softmax - $1/fan_{avg}$<br>
He - ReLU 함수와 그 변종들 - $2/fan_{avg}$<br>
르쿤 - SELU - $1/fan_{in}$

케라스는 기본적으로 균등분포(U($-\sqrt{3\sigma^2},\sqrt{3\sigma^2}$))의 글로럿 초기화를 사용

In [ ]:
# 다음과 같이 층을 바꿀때 초기화 전략을 바꿀 수 있음, 'he_normal' 대신 케라스 객체로 바꿔 쓸 수 있음
keras.layers.Dense(10,activation='rele',kernel_initializer='he_normal')

# He 초기화 전략을 사용하되 fan-in 대신 fan-avg을 쓰고 싶다면 다음과 같이
he_avg_init = keras.initializers.VarianceScaling(scale=2.,mode='fan_avg',distribution='uniform')
keras.layers.Dense(10,activation='sigmoid', kernel_initializer=he_avg_init)

## 기본적인 것 외 여러 활성화 함수의 종류
- LeakyReLU : max(${\alpha}z,z$)
- RRelu : 주어진 범위에서 $\alpha$를 무작위로 선택하고 테스트시에 평균을 사용
- PRelu : $\alpha$가 훈련하는 동안 학습됨.(다른 하이퍼파라미터처럼 역전파에 의해)
- ELU : z<0 일 때 $\alpha(exp(z)-1)$, z>0일 때는 z
- SELU : 스케일이 조정된 ELU 함수

 등등

In [ ]:
# LeakyReLU 활성화 함수를 사용하고자 한다면 LeakyReLU 층을 만들고 모델에서 적용하려는 층 뒤에 추가

model - keras.model.Sequential([
                                [...]
                                keras.layers.Dense(10,kernel_initializer='he_normal'),
                                keras.layers.LeakyReLU(alpha=.2),
                                [...]
])

# PReLU를 사용하기 위해서는 LeakyReLU(alpha=.2)를 PReLU()로 바꾼다.
# RReLU는 케라스에서 지원하지 않기에 직접 구현

# 배치 정규화
ELU 혹은 ReLU의 변종 함수와 함께 He 초기화를 사용하면 훈련 초기 단계에서 그레이디언트 소실이나 폭주 문제를 크게 감소시킬 수 있지만, 훈련 과정 중 재발하지 않으리라는 보장이 없다.

BN(Batch Normalization)(배치 정규화) 기법을 통해 이 문제를 해결할 수 있다.

이 기법은 각 층에서 활성화 함수를 통과하기 전이나 후에 모델에 연산을 하나 추가한다.  입력을 원점에 맞추고 정규화한 다음, 각 층에서 두 개의 새 파라미터로 결과값의 스케일을 조정하고 이동시킨다. 하나는 '스케일 조정', 또 하나는 '이동'에 사용한다. <br>
많은 경우 신경망의 첫 째 층에 배치 정규화를 추가하면 훈련 세트를 표준화(StandardScaler 등)할 필요가 없다.

입력 데이터를 원점에 맞추고 정규화하기 위해서는 평균과 표준편차를 추정해야한다.
- ${\mu}_B = {1\over{m_B}}{\sum_{i=1}^{m_B}}x^{(i)}$
- ${\sigma_B}^2 = {1 \over m_B}\sum_{i=1}^{m_B}(x^{(i)}-\mu_B)^2$
- $\hat{x}^{(i)} = {x^{(i)}-\mu_B \over \sqrt{\sigma_B^2 + \epsilon}}$ ($\epsilon$은 분모가 0이되는 것 방지)
- $z^{(i)} = \gamma*\hat{x}^{(i)} + \beta $   (*:elelmet-wise multiplication)

$\gamma$ : 층의 출력 파리미터 벡터, $\beta$ : 층의 출력 이동(오프셋) 파라미터 벡터,<br> $\hat{x}^{(i)}$:평균이 0이고 정규화된 샘플 i의 입력 ,$z^{(i)}$ : 배치 정규화 연산의 출력

테스트 세트의 경우 샘플이 하나씩 주입된다고 가정하기 때문에 입력의 평균과 표준편차를 계산할 방법이 없다. 훈련이 끝난 후 전체 훈련 세트를 신경망에 통과시켜 배치 정규화 층의 각 입력에 대한 평균과 표준편차를 대신 사용하는 것이 하나의 방법인데 대부분의 배치 정규화 구현은 층의 입력 평균과 표준편차의 이동 평균을 사용해 훈련하는 동안 훈련하는 동안 최종 통계를 수정한다. keras의 Batch Normalization 층은 이를 자동으로 수정한다.

정리하자면 다음 네 개의 파라미터가 필요한데 $\gamma$과 $\beta$는 일반적인 역전파를 통해 학습.<br>
$\mu$(최종 입력 평균 벡터)와 $\sigma$(최종 입력 표준편차)는 지수 이동 평균을 사용하여 추정된다. $\mu$와 $\sigma$는 훈련하는 동안 추정되지만 훈련이 끝난 후 사용된다.

# 케라스로 배치 정규화 구현

In [5]:
# 은닉층의 활성화 함수 전이나 후에 BatchNormalization 층을 추가

model = keras.models.Sequential([
                                 keras.layers.Flatten(input_shape=[28,28]),
                                 keras.layers.BatchNormalization(),
                                 keras.layers.Dense(300,activation='elu',kernel_initializer='he_normal'),
                                 keras.layers.BatchNormalization(),
                                 keras.layers.Dense(100,activation='elu',kernel_initializer='he_normal'),
                                 keras.layers.BatchNormalization(),
                                 keras.layers.Dense(10,activation='softmax')
                                 ])

In [6]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_1 (Flatten)          (None, 784)               0         
_________________________________________________________________
batch_normalization (BatchNo (None, 784)               3136      
_________________________________________________________________
dense (Dense)                (None, 300)               235500    
_________________________________________________________________
batch_normalization_1 (Batch (None, 300)               1200      
_________________________________________________________________
dense_1 (Dense)              (None, 100)               30100     
_________________________________________________________________
batch_normalization_2 (Batch (None, 100)               400       
_________________________________________________________________
dense_2 (Dense)              (None, 10)                1

배치 정규화 층은 입력마다 네 개의 파라미터 $\gamma,\beta,\mu,\sigma$를 추가한다.<br>ex)첫 째 층 : 4x28x28 = 3136
<br>이 중 절반은 역전파로 학습(업데이트)되지 않는 'Non-trainable' 파라미터

In [7]:
[(var.name, var.trainable) for var in model.layers[1].variables]

[('batch_normalization/gamma:0', True),
 ('batch_normalization/beta:0', True),
 ('batch_normalization/moving_mean:0', False),
 ('batch_normalization/moving_variance:0', False)]

In [8]:
# 활성화 함수 이전에 배치 정규화 층을 추가하기

model = keras.models.Sequential([
                                 keras.layers.Flatten(input_shape=[28,28]),
                                 keras.layers.BatchNormalization(),
                                 keras.layers.Dense(300,kernel_initializer='he_normal',use_bias=False),
                                 keras.layers.BatchNormalization(),
                                 keras.layers.Activation('elu'),
                                 keras.layers.Dense(100,kernel_initializer='he_normal'),
                                 keras.layers.BatchNormalization(),
                                 keras.layers.Activation('elu'),
                                 keras.layers.Dense(10,activation='softmax')
                                 ])

In [9]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_2 (Flatten)          (None, 784)               0         
_________________________________________________________________
batch_normalization_3 (Batch (None, 784)               3136      
_________________________________________________________________
dense_3 (Dense)              (None, 300)               235200    
_________________________________________________________________
batch_normalization_4 (Batch (None, 300)               1200      
_________________________________________________________________
activation (Activation)      (None, 300)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 100)               30100     
_________________________________________________________________
batch_normalization_5 (Batch (None, 100)              

## BatchNormalization 클래스의 하이퍼파라미터
momentum: 지수 이동 평균을 업데이트할 때<br>
새로운 값 v가 주어지면 다음과 같이 이동 평균 $\hat{v}$을 업데이트한다.
- $\hat{v} $ <- $\hat{v}*$momentum + $v*$(1-momentum)
<br>적절한 모멘텀 값은 일반적으로 1에 가깝다. 예를 들면 0.9, 0.99, 0.999 등(데이터셋이 크고 미니배치가 작으면 소수점 뒤에 9를 더 넣어 1에 가깝게 한다.)

axis: 매개변수를 정규화할 축을 결정한다. 기본값은 -1.

배치 정규화는 훈련 도중과 훈련이 끝난 후에 수행하는 계산이 다르다. 훈련 중간에는 배치 통계를 사용하고 훈련이 끝난 후에는 '최종' 통계를 사용한다.(이동 평균의 마지막 값)


# 그레이디언트 클리핑
폭주 문제를 해결하는 또 다른 방법은 역전파될 때 일정 임계값을 넘어서지 못하게 그레이디언트를 잘라내는 것이다. (==gradient clipping)<br>
RNN의 경우 배치 정규화를 적용하기 어려워 이 방법을 많이 사용한다.

In [10]:
# 케라스에서 이를 구혀하기 위해서는 옵티마이저를 만들때 clipvalue와 clipnorm 매개변수를 지정한다.

# 다음 옵티마이저는 그레이디언트 벡터의 모든 원소를 -1과 1 사이로 클리핑한다. 즉 손실의 모든 편미분 값을 -1과 1로 잘라낸다. 
#[0.9,100]->[0.9,1.0] : 벡터의 방향이 바뀐다.
optimizer = keras.optimizers.SGD(clipvalue=1.0)
model.compile(loss='mse',optimizer=optimizer)

# 클리핑이 그레이디언트 벡터의 방향을 바꾸지 않게 하려면 clipnorm을 지정해 노름으로 클리핑해야한다.
# [0.9,100.0]->[0.00899964,0.9999595]

# 두 매개변수가 모두 적용되면 clipnorm이 먼저 적용된다.

# 사전훈련된 층 재사용
대형 데이터셋을 처음부터 새로 훈련하는 것보다는 비슷한 유형의 문제를 처리한 신경망이 이미 있는지 찾아본 다음, 그 신경망의 하위층을 재사용하는 것이 좋다. 이를 전이학습(transfer learning)이라고 한다.

보통 새 작업에 가장 유용하지 못하고 필요 출력 개수에 맞지도 않은 원본 모델의 출력층을 바꿔야한다. 작업이 비슷할 수록 낮은 층부터 시작해 더 많은 층을 재사용하는 것이 좋다.

먼저 재사용하는 층을 모두 동결한다.(즉, 경사 하강법으로 가중치가 바뀌지 않도록 훈련되지 않는 가중치로 만든다.) 그다음 모델을 훈련하고 성능을 평가한다.<br>
맨 위에 있는 한두개의 은닉층의 동결을 해제하고 역전파를 통해 가중치를 조정하여 성능이 향상되는지 확인한다. <br>
훈련데이터가 많을수록 많은 층의 동결을 해제할 수 있다. 재사용 층의 동결을 해제할 때는 학습률을 줄이는 것이 좋다. 가중치를 세밀하게 튜닝하는데 도움이 된다.

만일 성능이 계속 좋지 않고 훈련 데이터가 적다면 상위 은닉층(들)을 제거하고 남은 은닉층을 동결하도록 한다. 

In [35]:
# fashion mnist를 분류해 89%의 성능을 낸 모델을 재활용하기로 한다.

model_A = keras.models.load_model('model_0916.h5')

In [37]:
model_A.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 784)               0         
_________________________________________________________________
dense (Dense)                (None, 300)               235500    
_________________________________________________________________
dense_1 (Dense)              (None, 100)               30100     
_________________________________________________________________
dense_2 (Dense)              (None, 10)                1010      
Total params: 266,610
Trainable params: 266,610
Non-trainable params: 0
_________________________________________________________________


In [36]:
# 샌들과 셔츠 이미지를 구분하는 작업 B를 해결하기 위해 이진분류기를 훈련하려한다.(양성 = 셔츠, 음성 = 샌들)

model_B_on_A = keras.models.Sequential(model_A.layers[:-1])
model_B_on_A.add(keras.layers.Dense(1,activation='sigmoid',name='dense_2'))

In [38]:
model_B_on_A.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 784)               0         
_________________________________________________________________
dense (Dense)                (None, 300)               235500    
_________________________________________________________________
dense_1 (Dense)              (None, 100)               30100     
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 101       
Total params: 265,701
Trainable params: 265,701
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# model_A와 model_B_on_A는 일부 층을 공유한다. 
# model_B_on_A를 훈련하면 model_A도 영향을 받는데 이를 원치 않으면 층을 재사용하기전 클론(clone)하도록 한다.

# 여기서 실제 실행은 하지 않음
model_A_clone = keras.models.clone_model(model_A)
model_A_clone.set_weights(model_A.get_weights())

model_B_on_A = keras.models.Sequential(model_A_clone.layers[:-1])
model_B_on_A.add(keras.layers.Dense(1,activation='sigmoid'))

In [7]:
from tensorflow.keras.datasets import fashion_mnist
fashion_mnist = keras.datasets.fashion_mnist
(X_train_full,y_train_full),(X_test,y_test) = fashion_mnist.load_data()

4423680/4422102 [==============================] - 0s 0us/step


In [22]:
# 셔츠와 슬리퍼만
X_train_B = X_train_full[(y_train_full==0)|(y_train_full==5)]
y_train_B = y_train_full[(y_train_full==0)|(y_train_full==5)]
X_test_B = X_test[(y_test==0)|(y_test==5)]
y_test_B = y_test[(y_test==0)|(y_test==5)]

In [24]:
# 스케일 조정
X_train_B = X_train_B/255.0
X_test_B = X_test_B/255.0

In [29]:
# 라벨링
y_train_B = np.where(y_train_B==0,1,0)
y_test_B = np.where(y_test_B==0,1,0)

In [30]:
from sklearn.model_selection import train_test_split
X_train_B,X_val_B,y_train_B,y_val_B = train_test_split(X_train_B,y_train_B)

In [39]:
# 새 출력층이 랜덤하게 초기화되어있으므로 큰 오차를 만들 것이다. 따라서 큰 오차 그레이디언트가 재사용된 가중치를 망칠 수 있으므로 
# 처음 몇 번의 에포크 동안 재사용된 층을 동결하고 새 층에서 적절한 가중치를 학습할 시간을 주는 것이다.
# 이를 위해 모든 층의 trainalbe 속성을 False로 지정하고 모델을 컴파일한다.

for layer in model_B_on_A.layers[:-1]:
  layer.trainable = False

model_B_on_A.compile(loss='binary_crossentropy',optimizer='sgd',metrics=['accuracy'])

# 층을 동결하거나 동결을 해제한 후 반드시 모델을 컴파일해야한다.

In [40]:
# 이제 몇 번의 에포크 동안 모델을 훈련할 수 있다. 그다음 재사용된 층의 동결을 해제하고 모델을 다시 컴파일한 후 작업 B에 맞게 세밀하게 튜닝하기 위해 훈련을 지속한다.
# 일반적으로 재사용된 층의 동결을 해제한 후 학습률을 낮추는 것이 좋다. 이렇게 하면 재사용된 가중치가 망가지는 것을 막아준다.

history = model_B_on_A.fit(X_train_B,y_train_B,epochs=4,validation_data = (X_val_B,y_val_B))

for layer in model_B_on_A.layers[:-1]:
  layer.trainable = True

optimizer = keras.optimizers.SGD(lr=1e-4) # 기본값은 1e-2
model_B_on_A.compile(loss='binary_crossentropy',optimizer=optimizer,
                     metrics=['acc'])
history = model_B_on_A.fit(X_train_B,y_train_B,epochs=16,
                           validation_data=(X_val_B,y_val_B))

Epoch 1/4
282/282 [==============================] - 1s 3ms/step - loss: 0.0888 - accuracy: 0.9883 - val_loss: 0.0297 - val_accuracy: 0.9990
Epoch 2/4
282/282 [==============================] - 1s 3ms/step - loss: 0.0252 - accuracy: 0.9980 - val_loss: 0.0182 - val_accuracy: 0.9990
Epoch 3/4
282/282 [==============================] - 1s 3ms/step - loss: 0.0183 - accuracy: 0.9980 - val_loss: 0.0139 - val_accuracy: 0.9990
Epoch 4/4
282/282 [==============================] - 1s 3ms/step - loss: 0.0150 - accuracy: 0.9982 - val_loss: 0.0116 - val_accuracy: 0.9993
Epoch 1/16
282/282 [==============================] - 1s 3ms/step - loss: 0.0138 - acc: 0.9980 - val_loss: 0.0115 - val_acc: 0.9993
Epoch 2/16
282/282 [==============================] - 1s 3ms/step - loss: 0.0138 - acc: 0.9980 - val_loss: 0.0114 - val_acc: 0.9993
Epoch 3/16
282/282 [==============================] - 1s 3ms/step - loss: 0.0137 - acc: 0.9980 - val_loss: 0.0113 - val_acc: 0.9993
Epoch 4/16
282/282 [====================

In [41]:
model_B_on_A.evaluate(X_test_B,y_test_B)

63/63 [==============================] - 0s 2ms/step - loss: 0.0131 - acc: 0.9980


[0.013086761347949505, 0.9980000257492065]

In [42]:
# 다음은 재사용 없이 그냥 20번의 학습을 시킨 모델, 구성은 똑같음
model_only_B = keras.models.Sequential()
model_only_B.add(keras.layers.Flatten(input_shape=[28,28]))
model_only_B.add(keras.layers.Dense(300,activation='relu'))
model_only_B.add(keras.layers.Dense(100,activation='relu'))
model_only_B.add(keras.layers.Dense(1,activation='sigmoid'))

model_only_B.compile(loss='binary_crossentropy',metrics=['acc'],optimizer='sgd')

model_only_B.fit(X_train_B,y_train_B,epochs=20,validation_data=(X_val_B,y_val_B))
print(model_only_B.evaluate(X_test_B,y_test_B))

Epoch 1/20
282/282 [==============================] - 1s 3ms/step - loss: 0.1521 - acc: 0.9657 - val_loss: 0.0440 - val_acc: 0.9960
Epoch 2/20
282/282 [==============================] - 1s 3ms/step - loss: 0.0363 - acc: 0.9944 - val_loss: 0.0225 - val_acc: 0.9970
Epoch 3/20
282/282 [==============================] - 1s 3ms/step - loss: 0.0246 - acc: 0.9960 - val_loss: 0.0166 - val_acc: 0.9970
Epoch 4/20
282/282 [==============================] - 1s 3ms/step - loss: 0.0193 - acc: 0.9968 - val_loss: 0.0138 - val_acc: 0.9977
Epoch 5/20
282/282 [==============================] - 1s 3ms/step - loss: 0.0164 - acc: 0.9971 - val_loss: 0.0117 - val_acc: 0.9980
Epoch 6/20
282/282 [==============================] - 1s 3ms/step - loss: 0.0141 - acc: 0.9972 - val_loss: 0.0120 - val_acc: 0.9973
Epoch 7/20
282/282 [==============================] - 1s 3ms/step - loss: 0.0127 - acc: 0.9972 - val_loss: 0.0093 - val_acc: 0.9993
Epoch 8/20
282/282 [==============================] - 1s 3ms/step - loss: 0.

데이터셋이 단순해 둘다 매우 높은 성과를 보이지만 초반 4번의 epoch를 살펴보면 된다.

사실 이번 같은 경우는 높은 성능을 가질 수 있게 타깃 클래스, 파라미터 등을 이미 정해 놓은 상태였기 때문에 속임수가 있다. 전이학습은 작은 완전 연결 네트워크에서는 잘 동작하지 않는데 작은 네트워크는 패턴 수를 적게 학습하고 완전 연결 네트워크는 특정 패턴을 학습하기 때문이다. <br>
전이 학습은 조금 더 일반적인 특성을 감지하는 경향이 있는 심층 합성곱 신경망에서 잘 동작한다.

## 비지도 사전훈련
이하 책의 내용 참고